In [76]:
import sys
print(sys.version)

3.13.9 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 19:09:58) [MSC v.1929 64 bit (AMD64)]


# Libraries

In [1]:
import pandas as pd

# Dataframe Preperation<br> 

Overview
- Rail Accident df stored at "df_accident"
- Population df stored at "df_population"

In [57]:
rail_accident_path = "./data/Rail_Equipment_Accident_Incident_Data_(Form_54)_20260116.csv"
population_path = "./data/Ruralurbancontinuumcodes2023.csv"

df_accident_all = pd.read_csv(
    rail_accident_path,
    low_memory = False
)
df_population = pd.read_csv(
    population_path,
    encoding="latin1"
)

In [58]:
print(df_accident_all.shape)
print(df_population.shape)

(223529, 155)
(9703, 5)


## Dataframe: Rail Accidents

Overview
- Year: int
- State Code: int
- County Code: float

In [59]:
print(df_accident_all.head(0))

Empty DataFrame
Columns: [Reporting Railroad Code, Reporting Railroad Name, Year, Accident Number, PDF Link, Accident Year, Accident Month, Other Railroad Code, Other Railroad Name, Other Accident Number, Other Accident Year, Other Accident Month, Maintenance Railroad Code, Maintenance Railroad Name, Maintenance Accident Number, Maintenance Accident Year, Maintenance Accident Month, Grade Crossing ID, Day, Date, Time, Accident Type Code, Accident Type, Hazmat Cars, Hazmat Cars Damaged, Hazmat Released Cars, Persons Evacuated, Subdivision, Division Code, Division, Station, Milepost, State Code, State Abbreviation, State Name, County Code, County Name, District, Temperature, Visibility Code, Visibility, Weather Condition Code, Weather Condition, Track Type Code, Track Type, Track Name, Track Class, Track Density, Train Direction Code, Train Direction, Equipment Type Code, Equipment Type, Equipment Attended, Train Number, Train Speed, Recorded Estimated Speed, Maximum Speed, Gross Tonnage

In [69]:
print(df_accident_all["Year"].dtype)
print(df_accident_all["State Code"].dtype)
print(df_accident_all["County Code"].dtype)

int64
int64
float64


### Grabbing items pass 2011

Accident Form updated at 2011

In [65]:
df_accident = df_accident_all[df_accident_all["Year"] > 2011]

In [62]:
print(df_pass_2011.head())

     Reporting Railroad Code                          Reporting Railroad Name  \
50                      BNSF                             BNSF Railway Company   
54                       CSX                               CSX Transportation   
56                       CSX                               CSX Transportation   
58                       CSX                               CSX Transportation   
1159                    SFRV  South Florida Regional Transportation Authority   

      Year Accident Number                                           PDF Link  \
50    2013       NW0813104  https://safetydata.fra.dot.gov/Officeofsafety/...   
54    2014       000131012  https://safetydata.fra.dot.gov/Officeofsafety/...   
56    2020       000191201  https://safetydata.fra.dot.gov/Officeofsafety/...   
58    2013       000121335  https://safetydata.fra.dot.gov/Officeofsafety/...   
1159  2024        03172024  https://safetydata.fra.dot.gov/Officeofsafety/...   

      Accident Year  Accid

In [44]:
df_accident[["Year", "State Code", "County Code"]].head()

,Year,State Code,County Code
50,2013,53,53.0
54,2014,21,15.0
56,2020,47,37.0
58,2013,45,47.0
1159,2024,12,99.0


## Dataframe: Population

Overview
- FIPS: int
- Value(population): object

**Update "Value" to int or float for comparison

In [70]:
print(df_population.head(0))

Empty DataFrame
Columns: [FIPS, State, County_Name, Attribute, Value]
Index: []


In [75]:
print(df_population["FIPS"].dtype)
print(df_population["Value"].dtype)

int64
object


# Data Cleaning

- FIPS (used to determine location) is 5 digits = State Code + County Code
- Find potential Null values

## Dataframe: Rail Accidents

Overview
- One Null "County Code"
- None Null "State Code"

In [55]:
df_accident["State Code"].isna().sum()

np.int64(0)

In [47]:
df_accident["County Code"].isna().sum()

np.int64(1)

In [66]:
df_accident = df_accident.dropna(subset=["County Code"])

In [67]:
df_accident["County Code"].isna().sum()

np.int64(0)

## Dataframe: Population

Overview
- None Null "FIPS"
- None Null "Value"

In [72]:
df_population["FIPS"].isna().sum()

np.int64(0)

In [74]:
df_population["Value"].isna().sum()

np.int64(0)

## Data Merging